**************************************************INITIALIZATION*********************************************************

In [ ]:
from tjmonopix.tjmonopix import TJMonoPix
import time

chip = TJMonoPix()
chip['CONF']['DEF_CONF_N'] = 0
chip['CONF'].write()
chip.init()
chip.power_on()

chip['CONF']['RESET_BCID'] = 1
chip['CONF']['RESET'] = 1
chip['CONF'].write()

chip['CONF']['EN_BX_CLK'] = 1
chip['CONF']['EN_OUT_CLK'] = 1
chip['CONF'].write()
 
chip['CONF']['RESET_BCID'] = 0
chip['CONF']['RESET'] = 0
chip['CONF'].write()

chip.default_conf()

#-------------------------------------------------#
chip['CONF_SR']['SET_ICASN'].setall(False)
chip['CONF_SR']['SET_ICASN'][64:64] = True

chip['CONF_SR']['SET_VRESET_P'].setall(False)
chip['CONF_SR']['SET_VRESET_P'][35] = 1 #1V

chip['CONF_SR']['SET_IRESET'].setall(False)
chip['CONF_SR']['SET_IRESET'][86:42] = True #1.5nA

chip['CONF_SR']['SET_ITHR'].setall(False)
chip['CONF_SR']['SET_ITHR'][71:56] = True #2.1nA

chip['CONF_SR']['EN_HV'].setall(False)
chip['CONF_SR']['EN_COMP'].setall(False)
chip['CONF_SR']['EN_PMOS'].setall(False)
chip['CONF_SR']['EN_PMOS_NOSF'].setall(False)
chip['CONF_SR']['EN_TEST_PATTERN'].setall(False)

chip['CONF_SR']['MASKD'].setall(False)
chip['CONF_SR']['MASKH'].setall(False)
chip['CONF_SR']['MASKV'].setall(False)

chip.write_conf()

chip['CONF']['DEF_CONF_N'] = 1
chip['CONF'].write()

****************************************************POWER_STATUS*********************************************************

In [ ]:
#chip['BiasSF'].set_current(100, unit='uA') # SET THE SOURCE FOLLOWER READOUT CURRENT
chip.get_power_status()

****************************ENABLE_COLUMS, SET MASKS, SELECT WHICH COLUMN AND ROW TO INJECT******************************

In [ ]:
# SELECT WHICH DOUBLE COLUMNS TO ENABLE
chip['CONF_SR']['EN_PMOS_NOSF'].setall(False)
chip['CONF_SR']['EN_PMOS'].setall(False)
chip['CONF_SR']['EN_COMP'].setall(False)
chip['CONF_SR']['EN_HV'].setall(False)
chip['CONF_SR']['EN_OUT'].setall(False) #ENABLES OR DISABLES THE NORMAL OUTPUT PADS, ACTIVE LOW
chip['CONF_SR']['nEN_OUT'].setall(True) #ENABLES OR DISABLES THE COMPLEMENTARY OUTPUT PADS, ACTIVE LOW
chip['CONF_SR']['EN_HITOR_OUT'].setall(True) #ENABLES OR DISABLES THE NORMAL HITOR PADS, HITOR0-3 =  1-4 flavor, ACTIVE LOW
chip['CONF_SR']['nEN_HITOR_OUT'].setall(True) #ENABLES OR DISABLES THE COMPLEMENTARY HITOR PADS, ACTIVE LOW

chip['CONF_SR']['EN_PMOS'][9] = 1
chip['CONF_SR']['EN_HITOR_OUT'][1] = 0

# SELECT WHICH PHYSICAL COLUMNS, ROWS, DIAGONALS TO MASK
# TO MASK ONE PIXEL, MASKV, MASKH and MASKD OF THIS PIXEL SHOULD BE 0 (FALSE)
# THE MASKD NUMBER OF THE PIXEL WE WANT TO MASK (or UNMASK), IS GIVEN BY: MASKD = PHYSCOL- PHYSROW
# IF PHYSCOL-PHYSROW<0, then MASKD = 448+PHYSCOL-PHYSROW
chip['CONF_SR']['MASKD'].setall(True)
chip['CONF_SR']['MASKH'].setall(True)
chip['CONF_SR']['MASKV'].setall(True)

# TO USE THE MASK FUNCTION YOU MUST INPUT THE FLAVOR, COLUMN AND ROW
# THE FLAVOR NUMERS IS: 0 FOR PMOS_NOSF, 1 FOR PMOS, 2 FOR COMP, 3 FOR HV
chip.mask(1,19,1)
chip.mask(1,18,166)
chip.mask(1,18,24)
chip.mask(1,18,200)

#chip['CONF_SR']['MASKD'][31] = True
#chip['CONF_SR']['MASKH'][99] = False

# SELECT WHICH PHYSICAL COLUMS TO INJECT
# INJ_IN_MON_L AND INJ_IN_MON_L SELECT THE LEFT AND RIGHT SPECIAL ANALOG MONITORING PIXELS
chip['CONF_SR']['COL_PULSE_SEL'].setall(False)
#chip['CONF_SR']['COL_PULSE_SEL'][130] = 1
chip.enable_column_injection(1,18)

chip['CONF_SR']['INJ_IN_MON_L'] = 1
chip['CONF_SR']['INJ_IN_MON_R'] = 0

# SELECT WHICH PHYSICAL ROWS TO INJECT
# THE SPEXIAL PIXELS OUTA_MON3 to OUTA_MON0 CORRESPONT TO ROWS 223 to 220 FOR INJECTION
chip['CONF_SR']['INJ_ROW'].setall(False)
chip['CONF_SR']['INJ_ROW'][99] = 1
chip['CONF_SR']['INJ_ROW'][223] = True

# SELECT PHYSICAL COLUMN(S) FOR HITOR OUTPUT
# nMASKH (SO SETTING MASKH TO FALSE) ENABLES HITOR FOR THE SPECIFIC ROW
chip['CONF_SR']['DIG_MON_SEL'].setall(False)
chip.enable_column_hitor(1,18)
#chip['CONF_SR']['DIG_MON_SEL'][130] = 1

chip.write_conf()

******************INITIALIZE COMMUNICATION, RECEIVE DATA (CHECK IF DATA ARE SENT WITH EACH EXCECUTION)*******************

In [ ]:
chip['data_rx'].CONF_STOP_FREEZE = 240 #default 40
chip['data_rx'].CONF_STOP = 245 #default 45

chip['data_rx'].set_en(True)

x = chip['fifo'].get_data()

ix = chip.interprete_data(x)
print(ix)

# ix = chip.interprete_raw_data(x)
# print(ix)

************CONFIGURE THE FRONT END CURRENT AND VOLTAGE BIASES, CONFIGURE THE HEIGHT OF THE INJECTION PULSE**************

In [ ]:
## SET THE INJECTION PULSE AMPLITUDE
## 128-bit DAC (7-bit binary equivalent)
## SET THE VOLTAGES IN ONE HOT ENCODING, ONLY ONE BIT ACTIVE AT A TIME.
## V = (127/1.8)*#BIT
# The default values are VL=44, VH=79, VH-VL=35
# VDAC LSB=14.17mV, Cinj=230aF, 1.43e-/mV, ~710e-
chip['CONF_SR']['SET_VL'].setall(False)
chip['CONF_SR']['SET_VL'][44] = 1
chip['CONF_SR']['SET_VH'].setall(False)
chip['CONF_SR']['SET_VH'][79] = 1

####### CONFIGURE THE FRONT END ######

# SET VRESET_P, THIS IS THE BASELINE OF THE FRONT END INPUT, ONE HOT ENCODING
chip['CONF_SR']['SET_VRESET_P'].setall(False)
chip['CONF_SR']['SET_VRESET_P'][35] = 1 #1V

## 128-bit DAC (7-bit binary equivalent)
## SET THE CURRENTS USING THERMOMETER ENCODING, I = #BITS_ACTIVE*140nA*SCALING, SCALING IS DIFFERENT FOR EACH CURRENT
## SCALING: IBIAS=10, IDB=16, ITHR=0.125, ICASN=4, IRESET=0.03125
## ACTIVE BITS SHOULD BE SET STARTING FROM THE MIDDLE e.g. for 15 active bits, (128-15)/2=56,5 so 56zeros,15ones,57zeros
## Thus, Ix[71:57] = True

# SET ICASN, THIS CURRENT CONTROLS THE OUTPUT BASELINE, BE CAREFUL NOT TO SET IT TO HIGH
# ALWAYS MONITOR THE POWER AFTER SETTING ICASN. IF VDDD IS SEVERAL mA, REDUCE IT UNTIL IT RETURNS TO 0
# ICASN MAINLY CONTROLS THE THRESHOLD
chip['CONF_SR']['SET_ICASN'].setall(False)
chip['CONF_SR']['SET_ICASN'][62:55] = True # approx 1.084V at -3V backbias, 600mV at 0V backbias

# SET IRESET, THIS CURRENT CONTROLS THE RESET RATE OF THE FRONT END INPUT (ALSO THE THRESHOLD)
chip['CONF_SR']['SET_IRESET'].setall(False)
chip['CONF_SR']['SET_IRESET'][65:57] = True #270pA
chip['CONF_SR']['SET_IRESET_BIT'] = 1 # 1 = HIGH LEAKAGE MODE, NORMAL SCALING, 0 = LOW LEAKAGE MODE, SCALING*0.01

# SET ITHR, THIS CURRENT CONTROLS THE RESET RATE OF THE OUTPUT (AND THE THRESHOLD)
chip['CONF_SR']['SET_ITHR'].setall(False)
chip['CONF_SR']['SET_ITHR'][60:56] = True #680pA

# SET ITHR, THIS CURRENT CONTROLS THE BIASING OF THE DISCRIMINATOR (AND THE THRESHOLD)
chip['CONF_SR']['SET_IDB'].setall(False)
chip['CONF_SR']['SET_IDB'][78:50] = True #500nA

# SET IBIAS, THIS CURRENT IS THE DC CURRENT OF THE MAIN BRANCH OF THE FRONT END THAT PROVIDES AMPLIFICATION
# IT CONTROLS MAINLY THE RISE TIME
chip['CONF_SR']['SET_IBIAS'].setall(False)
chip['CONF_SR']['SET_IBIAS'][82:41] = True #500nA

############ ENABLE THE DAC CURRENT MONITORING ###########
# chip['CONF_SR']['SWCNTL_DACMONI'] = 0

########## SET THE BIAS CURRENTS OF THE TWO STAGE SOURCE FOLLOWER THAT BUFFERS THE ANALOG MONITORING VOLTAGES #########
# CONTROLS THE RESPONSE TIME AND THE LEVEL SHIFT OF THE BUFFER
# chip['CONF_SR']['SET_IBUFN_L'] = 0b1001
# chip['CONF_SR']['SET_IBUFP_L'] = 0b0101

chip.write_conf()

****************************SET THE INJECTION PULSE PARAMETERS, INJECT x times and get the data**************************

In [ ]:
#chip['inj'].reset()
chip['inj'].set_delay(400)
chip['inj'].set_width(150)
chip['inj'].set_repeat(1)
chip['inj'].set_en(0)

while not chip['inj'].is_ready:
    time.sleep(0.001)

for _ in range(10):
    chip['inj'].is_ready

chip["inj"].start()

x = chip['fifo'].get_data()

ix = chip.interprete_data(x)
print(ix)

#ix = chip.interprete_raw_data(x)
#print(ix)

**********************************MAKE AN INJECTION SCAN -  S-CURVE OF ONE PIXEL*****************************************

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.optimize import curve_fit
from scipy.special import erf
import numpy as np

def scurve(x, A, mu, sigma):
    return 0.5 * A * erf((x - mu) / (np.sqrt(2) * sigma)) + 0.5 * A

class pixel:
    def __init__(self, col, row):
        self.col = col
        self.row = row
    
pix = pixel(col=18, row=99)
repeat=5000
start_dif=1
VHLrange=20
VL=50
cal=20.2

hits = np.zeros((VHLrange), dtype=int)
xhits = range(1,VHLrange+1)

###########################################################################
chip['CONF_SR']['SET_VL'].setall(False)
chip['CONF_SR']['SET_VL'][VL] = 1

for j in range(10):
    x = chip['fifo'].get_data()

for i in range(start_dif,VHLrange+1):
    chip['CONF_SR']['SET_VH'].setall(False)
    chip['CONF_SR']['SET_VH'][VL+i] = 1

    chip.write_conf()
    time.sleep(0.01)
    
    chip['inj'].set_delay(400)
    chip['inj'].set_width(150)
    chip['inj'].set_repeat(repeat)
    chip['inj'].set_en(0)
    
    while not chip['inj'].is_ready:
        time.sleep(0.001)

    for _ in range(10):
        chip['inj'].is_ready
        
    chip["inj"].start()
    
    x = chip['fifo'].get_data()

    ix = chip.interprete_data(x)
    
    #print(ix)
    
    cnt = 0
    for hit in ix:
        if hit['col'] == pix.col and  hit['row'] == pix.row and hit['noise'] == 0:
            cnt += 1
            
    hits[i-1] =  cnt
    
############################################################################################
#print hits
#print xhits

max_occ = repeat
threshold = (VHLrange)/2

popt, _ = curve_fit(scurve, xhits, hits, p0=[max_occ, threshold, 50.], check_finite=False)
#print popt

newx = np.arange(start_dif,VHLrange,0.01)
fit = scurve(np.arange(start_dif,VHLrange,0.01), *popt)

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15,5))
ax1.plot(xhits, hits, 'o', label='data')
ax1.plot(newx, fit, label='fit')
ax1.set(xlabel='DU', ylabel='Hit count',title=('ENC of pixel (' +str(pix.col) +',' +str(pix.row) +')'))
ax1.legend(loc='best')
ax1.text(0.7, repeat*0.8, ('Theshold=' +str(round(popt[1],2)) +'DU, ENC=' +str(round(popt[2],2)) +'DU'), bbox=dict(facecolor='white', alpha=0.5))
ax1.text(0.7, repeat*0.7, "1DU=14.1mV, 1,43e/mV", bbox=dict(facecolor='white', alpha=0.5))

hite = np.array(xhits)
hite = hite*cal

popt, _ = curve_fit(scurve, hite, hits, p0=[max_occ, threshold*cal, 400.], check_finite=False)
#print popt

newxe = np.arange(start_dif*cal,(VHLrange)*cal,0.01)
fite = scurve(np.arange(start_dif*cal,(VHLrange)*cal,0.01), *popt)

ax2.plot(hite, hits, 'o', label='data')
ax2.plot(newxe, fite, label='fit')
ax2.set(xlabel='e-', ylabel='Hit count',title=('ENC of pixel (' +str(pix.col) +',' +str(pix.row) +')'))
ax2.legend(loc='best')
ax2.text(15, repeat*0.8, ('Theshold=' +str(round(popt[1],2)) +'e-, ENC=' +str(round(popt[2],2)) +'e-'), bbox=dict(facecolor='white', alpha=0.5))
ax2.text(15, repeat*0.7, "1DU=14.1mV, 1,43e/mV", bbox=dict(facecolor='white', alpha=0.5))
fig.savefig('/faust/user/kmoustakas/ENC')